# <img style="float: left; padding-right: 10px; width: 150px" src="https://sc.acamica.com/icons/1j7w9h/social-300x300.png">  Acámica DS-COR3 - Ejercicio de Machine Learning + NLP

### Junio 2019

<hr style="height:2pt">

## Descripción

El objetivo de este notebook es ejercitar el modelado de un clasificador a partir de una fuente de datos de texto, en este caso referido a revisiones de películas de IMDB para analizar sentimientos positivos y negativos. Se debe procesar el texto "crudo" para extraer características numéricas, a partir de lo cual se modelará el clasificador utilizando una red neuronal simple mediante Keras.

**Recursos**:  
 
- https://towardsdatascience.com/sentiment-analysis-with-python-part-1-5ce197074184
- https://towardsdatascience.com/sentiment-analysis-with-python-part-2-4f71e7bde59a
- https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction
- https://www.kaggle.com/mikhailborovinskikh/tfidf-keras
- https://towardsdatascience.com/text-classification-in-keras-part-2-how-to-use-the-keras-tokenizer-word-representations-fd571674df23


In [1]:
# Dependencias
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import RMSprop
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import metrics

import nltk
import re

Using TensorFlow backend.


In [2]:
# Solo si es necesario descargar el corpus
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/leferrad/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Carga de datos

In [3]:
path_to_train_data = 'movie_data/full_train.txt'
path_to_test_data = 'movie_data/full_test.txt'

reviews_train_raw = [line.strip() for line in open(path_to_train_data, 'r')]
reviews_test_raw = [line.strip() for line in open(path_to_test_data, 'r')]

# datos ordenados por etiqueta (sentimiento positivo - negativo)
y_train = [1 if i < 12500 else 0 for i in range(25000)]
y_test = [1 if i < 12500 else 0 for i in range(25000)]

### Limpieza de datos

In [4]:
# Remover caracteres / tokens no deseados

REPLACE_NO_SPACE = re.compile("[.;:!\'?,\"()\[\]]")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

def preprocess_reviews(reviews):
    reviews = [REPLACE_NO_SPACE.sub("", line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(" ", line) for line in reviews]
    
    return reviews

reviews_train = preprocess_reviews(reviews_train_raw)
reviews_test = preprocess_reviews(reviews_test_raw)

In [5]:
# Remover palabras no deseadas

english_stop_words = nltk.corpus.stopwords.words('english')

def remove_stop_words(corpus):
    removed_stop_words = []
    for review in corpus:
        removed_stop_words.append(
            ' '.join([word for word in review.split() 
                      if word not in english_stop_words])
        )
    return removed_stop_words

reviews_train = remove_stop_words(reviews_train)
reviews_test = remove_stop_words(reviews_test)

In [6]:
reviews_train

['bromwell high cartoon comedy ran time programs school life teachers 35 years teaching profession lead believe bromwell highs satire much closer reality teachers scramble survive financially insightful students see right pathetic teachers pomp pettiness whole situation remind schools knew students saw episode student repeatedly tried burn school immediately recalled high classic line inspector im sack one teachers student welcome bromwell high expect many adults age think bromwell high far fetched pity isnt',
 'homelessness houselessness george carlin stated issue years never plan help street considered human everything going school work vote matter people think homeless lost cause worrying things racism war iraq pressuring kids succeed technology elections inflation worrying theyll next end streets given bet live streets month without luxuries home entertainment sets bathroom pictures wall computer everything treasure see like homeless goddard bolts lesson mel brooks directs stars bo

### Extracción de características

In [7]:
# **EJERCICIOS**:
# 1. Jugar con max_features: cuánto influye en los resultados de clasificación?
# 2. Probar CountVectorizer y comparar resultados con TF-IDF

# Basada en TF-IDF

# Aqui, jugar con los parámetros de TfidfVectorizer
max_features = 512

tfidf_vectorizer = TfidfVectorizer(max_features=max_features)
tfidf_vectorizer.fit(reviews_train)

X_train = tfidf_vectorizer.transform(reviews_train)
X_test = tfidf_vectorizer.transform(reviews_test)

In [8]:
# Visualizar un vector de ejemplo
X_train[0,:].todense()

matrix([[0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.18676207, 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.15039426, 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0

### Modelado

In [9]:
# Split en Train y Valid
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size = 0.7)

/home/leferrad/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [10]:
# Primero se define la ARQUITECTURA

# **EJERCICIOS**:
# 1. Jugar con DropOut: hasta cuánto ayuda / perjudica el entrenamiento?
# 2. Jugar con BatchNormalization: agregar estas capas supone una mejora?
# 3. Probar otras funciones de activación (excepto la salida que debe estar entre 0 y 1)
# 4. Probar otros tamaños de capas y cantidad de capas

model = Sequential()

model.add(Dense(256, activation='relu', input_shape=(max_features,)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())

model.add(Dense(1, activation='sigmoid'))

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
batch_normalization_2 (Batch (None, 128)               512       
_________________________________________________________________
dropout_2 (Dropout)  

In [11]:
# Luego definimos la OPTIMIZACIÓN

# **EJERCICIOS**
# 1. Probar RMSprop con parámetros por defecto (lr mayor, rho menor) y comparar resultados
# 2. Jugar con el batch_size (desde 10 hasta 1000) y entender cambios
# 3. Definir una cant. de epocas ideal para una config, en base al progreso del loss en train y valid

batch_size = 256
epochs = 100

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=1e-4, rho=0.95),
              metrics=['accuracy'])

In [12]:
# Ajustar el modelo!
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(X_val, y_val))

Instructions for updating:
Use tf.cast instead.
Train on 17500 samples, validate on 7500 samples
Epoch 1/100
17500/17500 [==============================] - 1s 78us/step - loss: 0.8359 - acc: 0.5288 - val_loss: 0.6965 - val_acc: 0.5644
Epoch 2/100
17500/17500 [==============================] - 1s 34us/step - loss: 0.7578 - acc: 0.5664 - val_loss: 0.6522 - val_acc: 0.6013
Epoch 3/100
17500/17500 [==============================] - 1s 38us/step - loss: 0.6984 - acc: 0.6029 - val_loss: 0.6080 - val_acc: 0.6531
Epoch 4/100
17500/17500 [==============================] - 1s 39us/step - loss: 0.6652 - acc: 0.6273 - val_loss: 0.5668 - val_acc: 0.6959
Epoch 5/100
17500/17500 [==============================] - 1s 38us/step - loss: 0.6156 - acc: 0.6711 - val_loss: 0.5254 - val_acc: 0.7341
Epoch 6/100
17500/17500 [==============================] - 1s 41us/step - loss: 0.5681 - acc: 0.7093 - val_loss: 0.4873 - val_acc: 0.7632
Epoch 7/100
17500/17500 [==============================] - 1s 44us/step - l

Epoch 58/100
17500/17500 [==============================] - 1s 39us/step - loss: 0.1991 - acc: 0.9217 - val_loss: 0.4723 - val_acc: 0.8255
Epoch 59/100
17500/17500 [==============================] - 1s 37us/step - loss: 0.1938 - acc: 0.9226 - val_loss: 0.4774 - val_acc: 0.8244
Epoch 60/100
17500/17500 [==============================] - 1s 39us/step - loss: 0.1861 - acc: 0.9270 - val_loss: 0.4874 - val_acc: 0.8231
Epoch 61/100
17500/17500 [==============================] - 1s 37us/step - loss: 0.1778 - acc: 0.9271 - val_loss: 0.4944 - val_acc: 0.8235
Epoch 62/100
17500/17500 [==============================] - 1s 38us/step - loss: 0.1707 - acc: 0.9330 - val_loss: 0.5069 - val_acc: 0.8224
Epoch 63/100
17500/17500 [==============================] - 1s 43us/step - loss: 0.1645 - acc: 0.9362 - val_loss: 0.5132 - val_acc: 0.8232
Epoch 64/100
17500/17500 [==============================] - 1s 40us/step - loss: 0.1631 - acc: 0.9363 - val_loss: 0.5192 - val_acc: 0.8217
Epoch 65/100
17500/17500 [=

In [13]:
classification_metrics = {"precision": metrics.precision_score, 
                          "recall": metrics.recall_score,
                          "accuracy": metrics.accuracy_score, 
                          "f1": metrics.f1_score}

threshold = 0.5  # **EJERCICIO** Variar threshold para entender cambios en resultados

print("Metricas para TRAIN:")

y_pred = [int(y > threshold) for y in model.predict(X_train).squeeze()]
for k, fun in classification_metrics.items():
    print("%s: %.3f" % (k, fun(y_train, y_pred)))

print("\n-------------------\n")

print("Metricas para VALID:")

y_pred = [int(y > threshold) for y in model.predict(X_val).squeeze()]
for k, fun in classification_metrics.items():
    print("%s: %.3f" % (k, fun(y_val, y_pred)))

print("\n-------------------\n")
    
print("Metricas para TEST:")

y_pred = [int(y > threshold) for y in model.predict(X_test).squeeze()]
for k, fun in classification_metrics.items():
    print("%s: %.3f" % (k, fun(y_test, y_pred)))
    
# **EJERCICIO** Hubo overfitting? cómo mitigarlo?

Metricas para TRAIN:
precision: 1.000
recall: 0.998
accuracy: 0.999
f1: 0.999

-------------------

Metricas para VALID:
precision: 0.834
recall: 0.776
accuracy: 0.814
f1: 0.804

-------------------

Metricas para TEST:
precision: 0.836
recall: 0.779
accuracy: 0.813
f1: 0.807


Evaluar los resultados como buenos Data Scientists y entender mejoras posibles sobre todo el pipeline para lograr un mejor modelo de análisis de sentimientos. Mientras tanto, felicitaciones por los buenos resultados!

![GREAT](https://media2.giphy.com/media/XreQmk7ETCak0/giphy.gif)